In [ ]:
# !git pull

In [1]:
import malaya_speech.train as train
import malaya_speech

In [2]:
encoded = malaya_speech.char.encode('hello')
encoded

[106, 103, 110, 110, 113, 1]

In [3]:
malaya_speech.char.decode(encoded)

'hello<EOS>'

In [4]:
from glob import glob
import os

files = glob('../speech-bahasa/LibriSpeech/*/*/*/*.trans.txt')
len(files)

672

In [5]:
audios, texts = [], []

for file in files:
    
    p = os.path.split(file)[0]
    with open(file) as fopen:
        d = list(filter(None, fopen.read().split('\n')))
    
    for row in d:
        splitted = row.split()
        s = os.path.join(p, splitted[0] + '.flac')
        text = ' '.join(splitted[1:])
        if not os.path.exists(s):
            print(s, 'not exist')
            continue
        
        audios.append(s)
        texts.append(text)

    
len(audios), len(texts)

(31159, 31159)

In [6]:
from sklearn.model_selection import train_test_split

_, audios, _, texts = train_test_split(audios, texts, test_size = 10000)

In [7]:
from tqdm import tqdm

def generator():
    for i in tqdm(range(len(audios))):
        wav_data, sr = malaya_speech.load(audios[i])

        yield {
            'waveforms': wav_data.tolist(),
            'waveform_lens': [len(wav_data)],
            'targets': malaya_speech.char.encode(texts[i]),
            'raw_transcript': [texts[i]],
        }
        
generator = generator()

In [8]:
import os
import tensorflow as tf

os.system('rm training/data/*')
DATA_DIR = os.path.expanduser('training/data')
tf.gfile.MakeDirs(DATA_DIR)

In [9]:
shards = [{'split': 'train', 'shards': 99}, {'split': 'dev', 'shards': 1}]

In [10]:
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'librispeech')

  0%|          | 0/10000 [00:00<?, ?it/s]


INFO:tensorflow:Generating case 0.


100%|██████████| 10000/10000 [06:25<00:00, 25.92it/s]


INFO:tensorflow:Generated 10000 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`



INFO:tensorflow:Data shuffled.
